# models.nn

> Fill in a module description here

In [ ]:
#| default_exp models.nn

In [ ]:
#| export
import torch
import torch.nn as nn

In [ ]:
#| export
def create_model(model_type, model_params, embedding_dim, final_hidden):
    model = CombinedModel(model_type, model_params, embedding_dim, final_hidden)
    return model

class TrendBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(TrendBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class SeasonalityBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(SeasonalityBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class NBEATS(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers, num_blocks):
        super(NBEATS, self).__init__()
        self.trend_blocks = nn.ModuleList([TrendBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])
        self.seasonality_blocks = nn.ModuleList([SeasonalityBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])

    def forward(self, x):
        backcast, forecast = x, 0
        print(x.shape)
        for block in self.trend_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        for block in self.seasonality_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        return forecast

class NHiTSBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(NHiTSBlock, self).__init__()
        self.hidden = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                    [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.hidden:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class NHiTS(nn.Module):
    def __init__(self, input_size, output_size, num_blocks, num_hidden, num_layers):
        super(NHiTS, self).__init__()
        self.trend_blocks = nn.ModuleList([NHiTSBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks)])

    def forward(self, x):
        forecast = torch.zeros((x.size(0), self.trend_blocks[0].forecast_fc.out_features), device=x.device)
        for block in self.trend_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        return forecast

class EmbeddingNetwork(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EmbeddingNetwork, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

class YearNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super(YearNetwork, self).__init__()
        self.hidden = nn.Linear(input_dim, 25)
        self.output = nn.Linear(25, num_hidden)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        return self.output(x)

In [ ]:
#| export
class CombinedModel(nn.Module):
    def __init__(self, model_type, model_params, embedding_dim, final_hidden):
        super(CombinedModel, self).__init__()
        
        if model_type == 'nbeats':
            self.ts_model = NBEATS(**model_params)
        elif model_type == 'nhits':
            self.ts_model = NHiTS(**model_params)
        else:
            raise ValueError("Invalid model type. Expected 'nbeats' or 'nhits'.")
        
        self.month_net = EmbeddingNetwork(12, embedding_dim)  # Months from 1 to 12
        self.day_net = EmbeddingNetwork(31, embedding_dim)    # Days from 1 to 31
        self.hour_net = EmbeddingNetwork(24, embedding_dim)   # Hours from 0 to 23
        
        # Calculate the input size for the final layer
        ts_output_size = model_params['output_size']
        combined_input_size = embedding_dim * 3 + ts_output_size
        self.final_layer = nn.Sequential(
            nn.Linear(combined_input_size, final_hidden),
            nn.ReLU(),
            nn.Linear(final_hidden, 1)
        )

    def forward(self, x, month, day, hour):
        ts_output = self.ts_model(x)
        month_output = self.month_net(month.long()).squeeze(1)
        day_output = self.day_net(day.long()).squeeze(1)
        hour_output = self.hour_net(hour.long()).squeeze(1)
        
        combined_output = torch.cat((ts_output, month_output, day_output, hour_output), dim=1)
        final_output = self.final_layer(combined_output)
        return final_output